## Рекомендация статей с помощью тематических моделей

In [6]:
from collections import OrderedDict

arxiv_tokens = OrderedDict()
with open('data/arxiv_plain.txt', 'r') as f:
    for line in f:
        cur_tokens = line.split()
        arxiv_tokens[cur_tokens[0]] = cur_tokens[1:]
arxiv_titles = list(arxiv_tokens.keys())

In [7]:
len(arxiv_tokens)

43091

In [9]:
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary

arxiv_dictionary = Dictionary(list(arxiv_tokens.values()))
arxiv_corpus = [arxiv_dictionary.doc2bow(text) for text in list(arxiv_tokens.values())]

In [10]:
lda = LdaModel(arxiv_corpus, num_topics=30)

In [11]:
lda.num_topics

30

In [12]:
lda.get_document_topics(arxiv_corpus[0])

[(5, 0.08792581),
 (6, 0.44382215),
 (10, 0.029720737),
 (15, 0.06753212),
 (26, 0.06441488),
 (28, 0.29692042)]

In [13]:
import tqdm

theta = {}
for doc_title, doc_bow in tqdm.tqdm(zip(arxiv_titles, arxiv_corpus)):
    topic_vector = np.zeros(lda.num_topics)
    for topic_num, topic_prob in lda.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = topic_vector


43091it [03:12, 223.64it/s]


Тематический вектор статьи с номером 0704.0004:

In [14]:
theta['0704.0004']

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.08794806, 0.44381464, 0.        , 0.        , 0.        ,
       0.02972036, 0.        , 0.        , 0.        , 0.        ,
       0.067536  , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.0643954 , 0.        , 0.29690889, 0.        ])

Теперь для того, чтобы порекомендовать читателю близкие по смыслу статьи, достаточно выбрать метрику близости и сравнить вектор текущего документа (например, последнего прочитанного) с векторами всех остальных документов в коллекции. В качестве метрики близости можно использовать косинусную меру, евклидово расстояние, расстояние Хелингера и т.д.

In [20]:
from sklearn import metrics
import numpy as np

def cos_sim(first, second):
    return metrics.pairwise.cosine_similarity(first.reshape(1, -1), second.reshape(1, -1))[0][0]

def dot_sim(first, second):
    return first.dot(second)

def hel_sim(first, second): #one more sqrt and division by sqrt(2) omitted, minus added
    return -np.sum((np.sqrt(first) - np.sqrt(second)) ** 2)

def jaccard_sim(first, second):
    intersection = set(first).intersection(set(second))
    union = set(first).union(set(second))
    return float(len(intersection))/float(len(union))

In [21]:
def recommend_papers(query, theta, sim=cos_sim, top_k=10):
    query_vec = theta[query]
    ranked_list = []
    for doc_name, doc_vec in theta.items():
        ranked_list.append((doc_name, sim(query_vec, doc_vec)))
    ranked_list.sort(key=lambda x: x[1], reverse=True)
    return ranked_list[:top_k]

In [22]:
recommended_papers = recommend_papers('0704.2596', theta, top_k=5)

In [23]:
from collections import Counter

for paper_name, prob in recommended_papers:
    print(paper_name)
    print(' '.join([token[0] for token in Counter(arxiv_tokens[paper_name]).most_common(10)]))
    print()

0704.2596
codes code minimum weight linear information using gr obner compute

0904.3148
bch long gf deg code parallel architecture polynomial lfsr encoding

1008.1498
matrix sparse null vector sparsification approximation problem space problems min

1111.4301
encryption homomorphic key our will scheme error every circuit kq

1401.1876
wg power optimal wch flow bus relaxation opf matrix r2



Для оценки качества полученной рекомендательной системы воспользуемся датасетом триплетов [[Dai et al. 2015](https://arxiv.org/abs/1507.07998)]. Датасет содержит тройки статей `<запрос>|<релевантная статья>|<нерелевантная статья>`. Будем считать, что если метрика близости между запросом и релевантной статьей оказалась выше, чем между запросом и нерелевантной статьей, то такая тройка обработана "правильно".

In [24]:
def evaluate_quality(theta, sim):
    all_triplets = 0
    covered_triplets = 0
    correct_triplets = 0
    with open('data/arxiv_triplets.txt', 'r') as fin:
        for line in fin:
            ids = list(map(lambda x: x.split('/pdf/')[-1], line.split()))
            if all([x in theta.keys() for x in ids]):
                covered_triplets += 1
                vectors = [theta[x] for x in ids]
                correct_triplets += sim(vectors[0], vectors[1]) > sim(vectors[0], vectors[2])
            all_triplets += 1

    return 1.0 * correct_triplets / covered_triplets

In [25]:
evaluate_quality(theta, cos_sim)

0.851889232321958

In [26]:
evaluate_quality(theta, hel_sim)

0.8640005046363465

In [27]:
evaluate_quality(theta, dot_sim)

0.8463382325111967

### Эксперимент №2: использовать BERT-based фичи совместно с тематическими фичами

In [ ]:
import transformers

BERT: http://jalammar.github.io/illustrated-bert/